In [ ]:
!apt-get install graphviz


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


## **Carregando os dados**

In [ ]:
# Importando bibliotecas necessárias
import numpy as np
import pandas as pd
from google.colab import files
from scipy.io import arff
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import graphviz

# Fazer upload do arquivo ARFF
uploaded = files.upload()

# O nome do arquivo é a chave no dicionário uploaded
filename = next(iter(uploaded))

# Carregar o arquivo ARFF
data, meta = arff.loadarff(filename)

# Converter para DataFrame do pandas
df = pd.DataFrame(data)

# Exibir os primeiros registros do DataFrame
print(df.head())

# Identificar as colunas de atributos e a coluna de classe
# (presumindo que a última coluna seja a coluna alvo, ajuste conforme necessário)
X = df.iloc[:, :-1].values  # Todas as colunas exceto a última (atributos)
y = df.iloc[:, -1].astype(str).values  # Última coluna (classe) convertida para string

# Normalizar os dados (StandardScaler padroniza os dados com média 0 e desvio padrão 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Normalizando todos os dados antes da validação cruzada

# Exibir os primeiros registros do DataFrame
print(X.view())

Saving mammographic-mass.arff to mammographic-mass.arff
    V1    V2   V3   V4        V5 Class
0  5.0  67.0  3.0  5.0  3.000000  b'2'
1  4.0  43.0  1.0  1.0  2.910734  b'2'
2  5.0  58.0  4.0  5.0  3.000000  b'2'
3  4.0  28.0  1.0  1.0  3.000000  b'1'
4  5.0  74.0  1.0  5.0  2.910734  b'2'
[[ 3.66084507e-01  7.97552241e-01  2.27914663e-01  1.44403739e+00
   2.44640809e-01]
 [-1.95635560e-01 -8.65089828e-01 -1.40884694e+00 -1.17704837e+00
  -1.16924460e-06]
 [ 3.66084507e-01  1.74061465e-01  1.04629546e+00  1.44403739e+00
   2.44640809e-01]
 ...
 [-1.95635560e-01  5.89721982e-01  1.04629546e+00  1.44403739e+00
   2.44640809e-01]
 [ 3.66084507e-01  7.28275488e-01  1.04629546e+00  1.44403739e+00
   2.44640809e-01]
 [-1.95635560e-01  4.51168476e-01  2.27914663e-01  1.33494510e-01
   2.44640809e-01]]


## **MLP com holdout sem repetição**

In [ ]:
# Dividir os dados em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)  # random_state=None para diferentes divisões

# Definir o modelo MLP
mlp = MLPClassifier(hidden_layer_sizes=(5,), activation='logistic', learning_rate='constant', learning_rate_init= 0.01, max_iter=1000, random_state=None)

# Treinar o modelo
mlp.fit(X_train, y_train)

# Fazer previsões
y_pred = mlp.predict(X_test)

# Avaliar a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

# Exibir a acurácia
print(f"Acurácia da MLP (sem validação cruzada e sem repetição): {accuracy * 100:.2f}%")

"""# Exibir a matriz de pesos sinápticos
for i, weights in enumerate(mlp.coefs_):
    print(f"Matriz de pesos da camada {i + 1}:\n{weights}\n")"""
"""
    # Função para desenhar a arquitetura da rede neural
def plot_mlp_architecture(mlp):
    dot = graphviz.Digraph()

    # Adicionar camada de entrada
    for i in range(X.shape[1]):
        dot.node(f'input_{i}', f'Input {i+1}', shape='circle')

    # Adicionar camadas ocultas
    for i in range(len(mlp.coefs_) - 1):
        for j in range(mlp.coefs_[i].shape[1]):
            dot.node(f'hidden_{i}_{j}', f'Hidden {i+1}-{j+1}', shape='circle')
            for k in range(mlp.coefs_[i].shape[0]):
                dot.edge(f'input_{k}', f'hidden_{i}_{j}')

    # Adicionar camada de saída
    for i in range(len(mlp.coefs_[-1])):
        dot.node(f'output_{i}', f'Output {i+1}', shape='circle')
        for j in range(mlp.coefs_[-2].shape[1]):
            dot.edge(f'hidden_{len(mlp.coefs_) - 2}_{j}', f'output_{i}')

    dot.attr(rankdir='LR')  # Direção do gráfico da esquerda para a direita
    return dot
    """

"""# Desenhar e exibir a arquitetura
architecture = plot_mlp_architecture(mlp)
architecture.render('mlp_architecture', format='png', cleanup=True)  # Salvar a imagem
architecture  # Exibir a imagem"""


Acurácia da MLP (sem validação cruzada e sem repetição): 81.87%


"# Desenhar e exibir a arquitetura\narchitecture = plot_mlp_architecture(mlp)\narchitecture.render('mlp_architecture', format='png', cleanup=True)  # Salvar a imagem\narchitecture  # Exibir a imagem"

# MLP com holdout com repetições

In [ ]:
# Definir o modelo MLP
mlp = MLPClassifier(hidden_layer_sizes=(5,), activation='logistic', learning_rate='constant', learning_rate_init= 0.01, max_iter=1000, random_state=None)

# Inicializar listas para armazenar as acurácias
accuracies = []

# Realizar Holdout com 30 repetições
for i in range(30):
    # Dividir os dados em treino (80%) e teste (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)  # random_state=None para diferentes divisões

    # Treinar o modelo
    mlp.fit(X_train, y_train)

    # Fazer previsões
    y_pred = mlp.predict(X_test)

    # Avaliar a acurácia do modelo
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calcular a média e o desvio padrão das acurácias
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

# Exibir os resultados
print(f"Acurácia média do MLP (Holdout com 30 repetições): {mean_accuracy * 100:.2f}%")
print(f"Desvio padrão da acurácia: {std_accuracy * 100:.2f}%")

Acurácia média do MLP (Holdout com 30 repetições): 83.32%
Desvio padrão da acurácia: 2.13%


## **MLP com validação cruzada**

In [ ]:
# Definir o modelo MLP
mlp = MLPClassifier(hidden_layer_sizes=(5,), activation='logistic', learning_rate='constant', learning_rate_init= 0.01, max_iter=1000, random_state=None)

# Realizar validação cruzada com 10 grupos
cv_scores = cross_val_score(mlp, X, y, cv=10)

# Calcular a média e o desvio padrão das acurácias
mean_accuracy = np.mean(cv_scores)
std_accuracy = np.std(cv_scores)

# Exibir os resultados
print(f"Acurácia média do MLP (Validação Cruzada com 10 grupos): {mean_accuracy * 100:.2f}%")
print(f"Desvio padrão da acurácia: {std_accuracy * 100:.2f}%")

Acurácia média do MLP (Validação Cruzada com 10 grupos): 83.04%
Desvio padrão da acurácia: 4.11%
